# 이미지 분류하기
합성곱 신경망을 이해하려면 먼저 합성곱 연산과 교차 상관 연산에 대해 알아야 한다.  
앞으로 합성곱 연산을 합성곱, 교차상관 연산을 교차상관이라고 줄여 부르겠다.  

</br>

## 합성곱 이해하기

두 배열 x와 w가 있다.  
이 중 원소수가 적은 배열 w의 원소 순서를 뒤집는다.  
이 뒤집은 w를 w^r이라고 한다.  
w^r의 첫 부분을 x의 첫 부분과 맞춰 놓고 서로 곱한다.  
예를 들어 아래와 같다고 가정해보자  
```python
 x = [2, 8, 3, 7, 1, 2, 0, 4, 5]
w^r= [3, 5, 1, 2]
```
이때 각 원소를 곱한 뒤 더한 식은 다음과 같다.  
```
2 * 3 + 9 * 5 + 3 * 1 + 7 * 2 = 63
```
즉, 첫 번째 합성곱의 결과는 63이다.  
  
w^r을 오른쪽으로 한 칸 이동해서 같은 방식으로 곱한 뒤 더한다.  
이때 얻은 값은 48이다.  
</br>

같은 방식으로 끝에 도달할 때까지 반복한다.  
이 때, 얻는 값은 차례대로  
63, 48, 49, 28, 21, 20이다.  
이와 같은 합성곱 수식은 x * w와 같이 표기한다.  
기존의 곱하기와 혼동할 수 있음을 주의  
</br>





## 합성곱 구현하기
넘파이로 구현할 수 있다.  

In [2]:
# 예시 배열 만들고 합성곱 구현하기
import numpy as np
w = np.array([2, 1, 5, 3])
x = np.array([2, 8, 3, 7, 1, 2, 0, 4, 5])
w_r = np.flip(w)      # filp 메서드로 간단하게 뒤집을 수 있다. 
print(w_r)

[3 5 1 2]


물론 파이썬의 슬라이스 연산자로 뒤집을 수도 있다.  
```python
w_r = w[::-1]
print(w_r)
# 결과: [3 5 1 2]
```

In [3]:
# 넘파이의 점곱으로 합성곱 수행
for i in range(6):
    print(np.dot(x[i:i+4], w_r))

63
48
49
28
21
20


## 싸이파이로 합성곱 수행하기
싸이파이는 합성곱을 위한 함수 convolve를 제공한다.  

In [4]:
from scipy.signal import convolve
convolve(x, w, mode='valid')

array([63, 48, 49, 28, 21, 20])

## 합성곱 신경망은 진짜 합성곱을 사용하지 않는다.
사실 대부분의 딥러닝 패키지들은 합성곱 신경망을 만들 때  
합성곱이 아닌 교차상관을 사용한다.  
교차상관에 대해 알아보자  
</br>

교차상관은 합성곱과 동일한 방법으로 연산이 진행되지만  
'미끄러지는 배열을 뒤집지 않는다' 는 점이 다르다.  
  
그러니까 선술한 합성곱에서 뒤집는 과정만 없는 것이다.  
</br>

마찬가지로 교차상관은 싸이파이의 correlate() 함수를 사용하여  
구현할 수 있다.  

In [ ]:
# 싸이파이로 교차상관 구현하기
from scipy.signal import correlate
correlate(x, w, mode='valid')

그럼 왜 합성곱 대신 교차상관을 쓰는 걸까?  
모델 훈련시 가중치 배열을 초기화 하는 과정을 생각해보자
```
모델 훈련과정 간단정리
1. 가중치를 무작위 값으로 초기화
2. 모든 샘플에 대해 정방향과 역방향 계산을 수행하여 가중치를 조금씩 학습(업데이트) 한다.
```
'미끄러지는 배열'이 가중치 배열에 해당한다.  
가중치 배열은 무작위로 초기화 되는데 따라서 가중치를 뒤집든 안 뒤집든  
딱히 상관이 없다.  
하지만 합성곱 신경망이라는 이름을 관례적으로 널리 사용하다보니  
그냥 이렇게 부르는 것이다.  

</br>
이제 합성곱 신경망의 핵심인 패딩과 스트라이드에 대해 알아보자  


# 패딩과 스트라이드
패딩은 원본 배열의 양 끝에 빈 원소를 추가하는 것이고  
스트라이드는 미끄러지는 배열의 간격을 조절하는 것이다.  
패딩은 밸리드, 풀, 세임으로 나뉘는데 순서대로 알아보자  

</br>

### 밸리드 패딩
자, 짧은 배열은 미끄러지면서 합성곱 연산에 참가하는데  
이때 긴 배열의 첫 번째 원소는 계산에 한 번만 참가하고 그 뒤의 원소들은  
그보다 더 많이 참가할 것이다.  
즉, 각 원소가 연산에 참여하는 정도가 다르다는 것이다. 
다음과 같이 구현할 수 있다.  
```python
correlate(x, w, mode='valid')
``` 

</br>
  
### 풀 패딩
풀 패딩은 모든 원소가 연산에 동일하게 참가시키기 위해 양 끝에  
가상의 원소를 추가한 것이다.  
이때 사용하는 가상의 원소 값은 0 이다.(때문에 제로패딩이라고도 한다.)  
다음과 같이 구현할 수 있다.  
```python
correlate(x, w, mode='full')
```

</br>

### 세임 패딩
세임 패딩은 긴 배열과 출력 배열의 길이가 같아지도록  
원본 배열의 앞부분에 제로패딩을 추가한 것이다.  
다음과 같이 구현할 수 있다.  
```python
correlate(x, w, mode='same')
```

</br>

### 스트라이드
스트라이드는 미끄러지는 간격을 조정한다.  
지금까지는 미끄러지는 간격이 1칸씩이었는데  
이걸 조정해서 2칸, 또는 그 이상으로 조절할 수 있다.  


# 2차원 배열에서의 합성곱
2차원 배열의 합성곱도 1차원 배열의 합성곱과 비슷하게 수행된다.  
아래와 같은 두 배열이 있다고 가정해보자  
```
x = 
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]

w = 
[2, 0]
[0, 0]
```
작은 배열의 큰 배열 부분에 합성곱을 시작한다.  
첫 번째는 x 배열의 다음과 같은 부분에 계산이 된다.  
```
x 배열의 일부분
[1, 2]
[4, 5]
```
이 부분과 w 배열의 합성곱은 다음과 같다.  
1 x 2 + 2 X 0 + 4 X 0 + 5 X 0  
  
다음 계산 배열은 다음 부분과 같다.  
```
x 배열의 2번째 계산 부분
[2, 3]
[5, 6]
```
이 부분을 하고 나면 오른쪽 끝에 도달했으므로 다음엔  
다시 맨 왼쪽 한 칸 아래를 계산하면 된다.  
즉, 계산횟수는 총 4번이다.  

</br>

싸이파이의 correlate2d() 함수를 통해 2차원 배열 합성곱을 계산해보자  

In [6]:
x = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])
w = np.array([[2, 0], [0, 0]])

from scipy.signal import correlate2d

correlate2d(x, w, mode='valid')

array([[ 2,  4],
       [ 8, 10]])

2차원 배열의 패딩과 스트라이드도 알아보자  
세임패딩의 경우, 배열의 오른쪽과 아랫쪽에 제로 패딩이 추가된다.  
세임패딩을 사용했으므로 원본 배열 크기과 같은 출력 배열이 만들어 진다.  

</br>

스트라이드의 경우, 미끄러지는 방향은 유지한 체 간격만 조정된다.  


# 텐서플로로 합성곱을 수행
지금까지 싸이파이를 사용했는데 당연히 텐서플로에도 이 기능이 있다.  
텐서플로에서 2차원 합성곱 합수는 conv2d()이다.  
여기에 4차원 배열을 입력해야 한다.  
그 이유는 입력 이미지의 높이와 너비 외에 더 많은 차원이  
필요하기 때문이다.  